# 提示词模板之ChatPromptTemplate

1、 实例化的方式（两种方式：使用构造方法、form_message()）

2、 调用提示词模板的几种方法：invoke()\format()\format_message()\format_prompt()

3、 丰富的实例化参数类似

4、 结合LLM

5、 插入消息列表：MessagePlaceholder

# 1、实例化的方法
构造方法创建 ChatPromptTemplate

In [3]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, MessagesPlaceholder

#  ------------ 构造方法创建
chat_prompt = ChatPromptTemplate(messages=[
    ("system","你是一个AI助手，你的名字叫{name}"),
    ("human","我的问题是{question}")
]
# , input_variables=[               #这个参数可有可无
#     "name","question",
# ]

)

response = chat_prompt.invoke(input={"name":"小智","question":"1+2*5="})
print(response)

messages=[SystemMessage(content='你是一个AI助手，你的名字叫小智', additional_kwargs={}, response_metadata={}), HumanMessage(content='我的问题是1+2*5=', additional_kwargs={}, response_metadata={})]


调用from_message() 方法创建ChatPromptTemplate

In [4]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

#  ------------ from_messages 创建  --------------------- 两个方式创建所需的参数都是一致的
chat_prompt = ChatPromptTemplate.from_messages(
    messages=[
    ("system","你是一个AI助手，你的名字叫{name}"),
    ("human","我的问题是{question}")
]
)

response = chat_prompt.invoke(input={"name":"小智","question":"1+2*5="})
print(response)

messages=[SystemMessage(content='你是一个AI助手，你的名字叫小智', additional_kwargs={}, response_metadata={}), HumanMessage(content='我的问题是1+2*5=', additional_kwargs={}, response_metadata={})]


# 2、调用提示词的几种方法
- invoke()      重点记忆
- format()
- format_message()
- format_prompt()

In [12]:
# ----------------format调用方式------------------
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    messages=[
    ("system","你是一个AI助手，你的名字叫{name}"),
    ("human","我的问题是{question}")
]
)

response = chat_prompt.format(name="小智",question="1+2*5=")
print(response)
print(type(response))

System: 你是一个AI助手，你的名字叫小智
Human: 我的问题是1+2*5=
<class 'str'>


In [13]:
# ----------------format_messages调用方式------------------
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    messages=[
    ("system","你是一个AI助手，你的名字叫{name}"),
    ("human","我的问题是{question}")
]
)

response = chat_prompt.format_messages(name="小智",question="1+2*5=")
print(response)
print(type(response))


[SystemMessage(content='你是一个AI助手，你的名字叫小智', additional_kwargs={}, response_metadata={}), HumanMessage(content='我的问题是1+2*5=', additional_kwargs={}, response_metadata={})]
<class 'list'>


In [16]:
# ----------------format_prompt调用方式------------------
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    messages=[
    ("system","你是一个AI助手，你的名字叫{name}"),
    ("human","我的问题是{question}")
]
)

response = chat_prompt.format_prompt(name="小智",question="1+2*5=")
print(response)
print(type(response))


# 将ChatPromptValue类型转换为消息构成的list
response_message = response.to_messages()
print(response_message)
print(type(response_message))

# 将ChatPromptValue类型转换为字符串类型
response_message = response.to_string()
print(response_message)
print(type(response_message))

messages=[SystemMessage(content='你是一个AI助手，你的名字叫小智', additional_kwargs={}, response_metadata={}), HumanMessage(content='我的问题是1+2*5=', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.prompt_values.ChatPromptValue'>
[SystemMessage(content='你是一个AI助手，你的名字叫小智', additional_kwargs={}, response_metadata={}), HumanMessage(content='我的问题是1+2*5=', additional_kwargs={}, response_metadata={})]
<class 'list'>
System: 你是一个AI助手，你的名字叫小智
Human: 我的问题是1+2*5=
<class 'str'>


## 3、更丰富的实例化参数类型（非重点）
本质：不管使用构造方法、还是使用from_message()来创建ChatPromptTemplate的实例，本质上来讲，传入的都是消息构成的列表。

从调用上来讲，我们看到，不管使用构造方法，还是使用from_message()，messages参数类型都是列表，但是列表元素的类型是多样的。可以是：
字符串类型、字典类型、消息类型、元组构成的列表、Chat提示词模板类型、消息提示词模板类型

## 4、结合LLM

In [17]:
# 1、 提供大模型
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import os
import dotenv

dotenv.load_dotenv()    # 加载当前目录下的 .env 文件
os.environ.setdefault("OPENAI_API_KEY", os.getenv("OPENAI_API_KEY"))
os.environ.setdefault("OPENAI_BASE_URL", os.getenv("OPENAI_BASE_URL"))

# 获取对话模型
model = ChatOpenAI(
    model="deepseek-chat",
    temperature=0.7,
    max_tokens=20
)


# 2、通过Chat提示词模板、创建提示词
chat_prompt = ChatPromptTemplate.from_messages(
    messages=[
    ("system","你是一个AI助手，你的名字叫{name}"),
    ("human","我的问题是{question}")
]
)
response = chat_prompt.invoke({"name":"小白","question":"1+2*5="})

# 3、通过大模型调用提示词，得到响应数据
result = model.invoke(response)
print(result)
print(type(result))

content='我们按照运算顺序来计算：  \n先算乘法 **2 × 5 = 10**，  \n然后加法 **1 + 10 = 11**。  \n\n所以结果是：  \n**1 + 2 × 5 = 11** ✅' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 21, 'total_tokens': 72, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 21}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '5ece9c8c-8396-476b-bc2e-7dc0c464d603', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c0e08-90f9-77d0-8e71-4aa877f31c97-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 21, 'output_tokens': 51, 'total_tokens': 72, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}
<class 'langchain_core.messages.ai.AIMessage'>


## 5、插入消息列表
- 占位符功能：在提示词模板中预留位置，在运行时可以用实际的消息列表
- 保持消息结构：能够确保插入的消息保持原有的消息类型，不会将这些消息转换成字符串。这样，模型可以区分不同角色发送的消息
- 灵活性：可以用于构建复杂的对话流程，例如，我们可以将整个对话历史、或者某些中间步骤的消息动态的放入提示中
- 使用场景
    - 多轮对话
    - 动态消息插入
当不确定消息提示模板使用什么角色，或者系统在格式化过程中插入消息列表时，该怎么办？这就需要使用MessagesPlaceholder，负责在特定位置添加消息列表

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import MessagesPlaceholder
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.ai import AIMessage

chat_prompt = ChatPromptTemplate.from_messages(
    messages=[
        ("system", "你是一个AI助手，你的名字叫{name}"),
        ("human", "我的问题是{question}")
    ]
)

# 当不确定使用system还是human的时候，我们可以使用MessagesPlaceholder
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个AI助手，你的名字叫{name}"),
    MessagesPlaceholder("msgs")
])

response = chat_prompt.invoke({
    "name": "小白",
    "msgs": [HumanMessage(content="我的问题是：1+2*5="), AIMessage(content="1+3*6=？")]
})
print(response)

E:\shangdj\python\env\conda\envs\myPython3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


messages=[SystemMessage(content='你是一个AI助手，你的名字叫小白', additional_kwargs={}, response_metadata={}), HumanMessage(content='我的问题是：1+2*5=', additional_kwargs={}, response_metadata={}), AIMessage(content='1+3*6=？', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


存储历史对话记录

In [2]:
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import MessagesPlaceholder
from langchain_core.messages.human import HumanMessage

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        MessagesPlaceholder("history"),
        ("human", "{question}")
    ])

prompt = chat_prompt.format_messages(
    history=[HumanMessage(content="我的问题是：1+2*3=？"), AIMessage(content="1+2*3=7")],
    question="我刚才的问题是什么？")


# 1、 提供大模型
from langchain_openai import ChatOpenAI
import os
import dotenv

dotenv.load_dotenv()    # 加载当前目录下的 .env 文件
os.environ.setdefault("OPENAI_API_KEY", os.getenv("OPENAI_API_KEY"))
os.environ.setdefault("OPENAI_BASE_URL", os.getenv("OPENAI_BASE_URL"))

# 获取对话模型
model = ChatOpenAI(
    model="deepseek-chat",
    temperature=0.7,
    max_tokens=20
)

result = model.invoke(prompt)
print(result)

content='你刚才的问题是：**“1+2*3=？”**  \n\n答案是 **7**（按照运算顺序：先乘除后加减，所以先算 2×3=6，再加 1 得到 7）。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 34, 'total_tokens': 82, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 34}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '1095c0f1-3e39-4ccd-ae10-2e2bb5a82484', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c1c2c-e559-7983-af56-7254cd04c6d8-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 34, 'output_tokens': 48, 'total_tokens': 82, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}
